In [2]:
import os
import pandas as pd

In [3]:
columns_map = {
    'Time': 'time',
    'Tempo de processo': 'process_time',
    'tempo de processo (s)': 'process_time',
    'Tempo de processo (s)': 'process_time',
    'PT100 1 Celsius': 'sensor_PT100_1',
    'PT100 2 Celsius': 'sensor_PT100_2',
    'PT100 3 Celsius': 'sensor_PT100_3',
    'PT100 4 Celsius': 'sensor_PT100_4',
    'Temp. TH 1 Celsius': 'sensor_TH_1',
    'Temp. TH 2 Celsius': 'sensor_TH_2',
    'Umidade 1 [%]': 'sensor_humidity_1',
    'Umidade 2 [%]': 'sensor_humidity_2'
}

In [11]:
def filter_df(df:pd.DataFrame, output_dir:str, verbose=False):
    refactored_dataframe = pd.DataFrame()
    df_columns = list(df.columns)

    for column in df_columns:
        if (column == 'Hora'): pass

        elif (column == 'Tempo (min)'):
            time_in_seconds = []

            for time_in_minutes in df[column]:
                number_time = float(time_in_minutes.replace(',', '.'))
                time_in_seconds.append(f'{(number_time * 60):.2f}')

            refactored_dataframe['process_time'] = pd.Series(time_in_seconds)

        else:
            mapped_column_name = columns_map[column]
            if (mapped_column_name == 'time'):
                try:
                    for i, time_in_minutes in enumerate(df[column]):
                        space_character_index = time_in_minutes.index(' ')
                        df[column][i] = time_in_minutes[space_character_index+1:]
                except: pass
            refactored_dataframe[mapped_column_name] = df[column]
    
    refactored_dataframe.to_csv(output_dir, sep=';', index=False)
    if (verbose): refactored_dataframe.head()
    return refactored_dataframe

In [12]:
directory = '../data/Dados-tratados'
df_different_columns = []

for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    exception_files = ['.DS_Store']

    if os.path.isfile(file_path) and filename not in '.DS_Store':
        df = pd.read_csv(file_path, delimiter=';', on_bad_lines='skip')
        # check if destiny dir exists, otherwise create it
        new_df = filter_df(df, f'./fixed/{filename}')
        for column in new_df.columns:
            if (column not in df_different_columns): df_different_columns.append(column)

print("Different columns:")
for column in df_different_columns:
    print(column)

Different columns:
time
process_time
sensor_PT100_1
sensor_PT100_2
sensor_PT100_3
sensor_PT100_4
sensor_humidity_1
sensor_TH_1
sensor_humidity_2
sensor_TH_2
